In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [21]:
import time
import json
import os
import random
from typing import Dict, Iterator, List, Tuple

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from loguru import logger
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import KFold

from nam.config.default import defaults
from nam.types import Config
from nam.utils.args import parse_args
from nam.data.base import NAMDataset
from nam.models.featurenn import FeatureNN
from nam.models.nam import NAM

from main import get_config, init_random_seeds

In [22]:
config = get_config()

init_random_seeds(config.seed)

2020-11-25 01:20:42.412 | DEBUG    | main:get_config:30 - Creating output dir `/Users/amrmkayid/Desktop/nam/output`
2020-11-25 01:20:42.413 | DEBUG    | main:get_config:35 - Creating run exp folder `/Users/amrmkayid/Desktop/nam/output/Exp_2020-11-25_012042`
2020-11-25 01:20:42.414 | INFO     | main:get_config:48 - Saving configuration file in `/Users/amrmkayid/Desktop/nam/config.json`
2020-11-25 01:20:42.416 | DEBUG    | main:init_random_seeds:63 - PyTorch is using CPU...


In [23]:
config

namespace(device=device(type='cpu'),
          output_dir='/Users/amrmkayid/Desktop/nam/output',
          lr=0.01,
          batch_size=1024,
          l2_regularization=0.0,
          output_regularization=0.0,
          decay_rate=0.995,
          dropout=0.5,
          feature_dropout=0.0,
          data_split=1,
          seed=1,
          num_basis_functions=1000,
          units_multiplier=2,
          num_units=64,
          cross_val=False,
          max_checkpoints_to_keep=1,
          save_checkpoint_every_n_epochs=10,
          n_models=1,
          num_splits=3,
          fold_num=1,
          activation='exu',
          shuffle=True,
          regression=False,
          debug=False,
          shallow=False,
          use_dnn=False,
          early_stopping_epochs=60,
          n_folds=5,
          training_epochs=None,
          learning_rate=0.01,
          dataset_name='Teleco',
          log_dir='/Users/amrmkayid/Desktop/nam/output/Exp_2020-11-25_012042/logs',
       

## GALLUP Data

In [24]:
features_columns = ["income_2", "WP1219", "WP1220", "weo_gdpc_con_ppp"]
targets_column = ["country"]
weights_column = ["wgt"]

In [25]:
dataset = NAMDataset(config=config,
                    csv_file='data/GALLUP.csv',
                    features_columns=features_columns,
                    targets_column=targets_column,
                    weights_column=weights_column)
dataset

In [26]:
data_loaders = dataset.data_loaders(
      n_splits=config.num_splits,
      batch_size=config.batch_size,
      shuffle=config.shuffle,
      stratified=not config.regression,
      random_state=config.seed,
)

In [31]:
model = NAM(
      config=config,
      name="NAMModel",
      num_inputs=len(dataset[0][0]),
      num_units=config.num_units,
      shallow=config.shallow,
      feature_dropout=config.feature_dropout,
).to(device=config.device)

In [33]:
model.__dict__

{'training': True,
 '_parameters': OrderedDict([('_bias', Parameter containing:
               tensor([0.], requires_grad=True))]),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_hooks': OrderedDict(),
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_modules': OrderedDict(),
 '_config': namespace(device=device(type='cpu'),
           output_dir='/Users/amrmkayid/Desktop/nam/output',
           lr=0.01,
           batch_size=1024,
           l2_regularization=0.0,
           output_regularization=0.0,
           decay_rate=0.995,
           dropout=0.5,
           feature_dropout=0.0,
           data_split=1,
           seed=1,
           num_basis_functions=1000,
           units_multiplier=2,
           num_units=64,
           cross_val=False,
           max_checkpoints_to_keep=1,
           save_checkpoint_every_n_epochs=10,
           n_models

In [35]:
train, val = next(iter(data_loaders))

Fold((2,)), train: 938566, test: 134081


In [37]:
batch = next(iter(train))
batch

[tensor([[17671,     0,     8,    27],
         [45648,     0,    41,   404],
         [95431,     1,    21,   977],
         ...,
         [10845,     1,    15,    38],
         [10675,     1,    14,   448],
         [27616,     0,    34,   286]]),
 tensor([[243450],
         [  8889],
         [ 96660],
         ...,
         [279584],
         [ 89668],
         [125071]]),
 tensor([[139],
         [108],
         [  6],
         ...,
         [139],
         [140],
         [150]])]

In [40]:
model(batch[0])

tensor([ -33163.3281,  -86730.2578, -181812.3438,  ...,  -20415.4629,
         -21435.3848,  -52597.6602], grad_fn=<AddBackward0>)

### K-fold

In [8]:
for train_dl, val_dl in dataset.data_loaders():
    print(next(iter(train_dl)), next(iter(val_dl)))

Fold((1,)), train: 938566, test: 134081
[tensor([[76845,     1,    40,   605],
        [80149,     0,    43,   861],
        [    0,     1,     4,   677],
        ...,
        [99347,     1,    44,   965],
        [79297,     0,    46,   940],
        [85594,     0,    24,   915]]), tensor([[  8883],
        [106160],
        [146743],
        ...,
        [ 12864],
        [ 77549],
        [  6516]]), tensor([[  8],
        [100],
        [ 62],
        ...,
        [ 50],
        [135],
        [ 68]])] [tensor([[ 30686,      0,     31,    389],
        [103458,      1,     47,   1040],
        [ 41558,      0,      5,    133],
        ...,
        [ 71748,      1,      6,    442],
        [ 75880,      0,     48,    965],
        [ 65557,      1,     20,    758]]), tensor([[ 54025],
        [233171],
        [ 57097],
        ...,
        [105269],
        [ 35618],
        [251079]]), tensor([[ 61],
        [133],
        [119],
        ...,
        [ 17],
        [ 50],
        [

In [12]:
batch = next(iter(train_dl))
batch[0].shape

torch.Size([1024, 4])

## Housing Data

In [4]:

features_columns = [
    "longitude", "latitude", "housing_median_age", "total_rooms",
    "total_bedrooms", "population", "households", "median_income"
]
targets_column = ["median_house_value"]

In [5]:
dataset = NAMDataset(config=config,
                    csv_file='data/housing.csv',
                    features_columns=features_columns,
                    targets_column=targets_column)
dataset

In [6]:
for train_dl, test_dl in dataset.data_loaders():
    print(next(iter(train_dl)), next(iter(test_dl)))

Fold((1,)), train: 16512, test: 4128
[tensor([[ 252,  583,   16,  ..., 2395, 1314, 9240],
        [ 484,  350,   33,  ...,  753,  296, 5622],
        [ 573,  124,   45,  ..., 1518,  586, 1051],
        ...,
        [ 330,  468,   26,  ..., 1435,  478, 3444],
        [ 201,  472,    9,  ..., 2002, 1000, 5214],
        [ 271,  570,   19,  ...,  907,  335, 8112]]), tensor([[1379],
        [ 328],
        [1334],
        ...,
        [ 276],
        [2098],
        [1775]])] [tensor([[  689,     4,    19,  ...,  1470,   352,  6979],
        [  677,    19,    51,  ...,   846,   533,  1960],
        [  631,   110,     1,  ...,  2820,   789, 12705],
        ...,
        [  563,   146,    39,  ...,   832,   229,   252],
        [  178,   468,    14,  ...,   952,   363, 12756],
        [  607,   120,     8,  ...,  1230,   164,  5482]]), tensor([[ 905],
        [ 710],
        [3786],
        ...,
        [ 930],
        [3841],
        [1237]])]
Fold((2,)), train: 16512, test: 4128
[tensor([[  

In [7]:
batch = next(iter(train_dl))
batch

[tensor([[  529,   165,    32,  ...,  1396,   469, 10207],
         [  684,    36,    15,  ...,   787,   290, 11128],
         [  185,   495,    25,  ...,  1374,   606,  3480],
         ...,
         [  595,   119,    21,  ...,  2351,   659, 11533],
         [  285,   616,    40,  ...,  1653,   709,  3520],
         [  463,   361,    26,  ...,  1151,   308,   555]]),
 tensor([[2064],
         [2127],
         [1425],
         ...,
         [2413],
         [ 974],
         [  75]])]

In [42]:
inputs = batch[0]
torch.chunk(inputs, 8, dim=-1)[0].shape

torch.Size([1024, 1])

In [43]:
inputs_tuple = torch.chunk(inputs, 8, dim=-1)
ind_out = [input_i for i, input_i in enumerate(inputs_tuple)]

In [50]:
stacked_out = torch.stack(ind_out, dim=-1)

In [53]:
torch.sum(stacked_out, dim=-1).shape

torch.Size([1024, 1])

In [9]:
batch[0].shape[-1], batch[1].shape

(8, torch.Size([1024, 1]))

In [20]:
nam = NAM(config=config, name="NAMModel", num_inputs=batch[0].shape[-1], num_units=64, shallow=False)

In [21]:
nam.feature_nns

[FeatureNN(
   (relu): ReLU(inplace=True)
   (dropout): Dropout(p=0.0, inplace=False)
   (_h1): Linear(in_features=64, out_features=64, bias=True)
   (_h2): Linear(in_features=64, out_features=32, bias=True)
   (linear): Linear(in_features=32, out_features=1, bias=True)
   (hidden_layers): Sequential(
     (0): ExU()
     (1): Linear(in_features=64, out_features=64, bias=True)
     (2): Linear(in_features=64, out_features=32, bias=True)
   )
 ),
 FeatureNN(
   (relu): ReLU(inplace=True)
   (dropout): Dropout(p=0.0, inplace=False)
   (_h1): Linear(in_features=64, out_features=64, bias=True)
   (_h2): Linear(in_features=64, out_features=32, bias=True)
   (linear): Linear(in_features=32, out_features=1, bias=True)
   (hidden_layers): Sequential(
     (0): ExU()
     (1): Linear(in_features=64, out_features=64, bias=True)
     (2): Linear(in_features=64, out_features=32, bias=True)
   )
 ),
 FeatureNN(
   (relu): ReLU(inplace=True)
   (dropout): Dropout(p=0.0, inplace=False)
   (_h1): Line

In [19]:
nam(batch[0])

tensor([-7615.4722, -8860.6367, -2092.2812,  ..., -8710.9600, -1141.4934,
          300.8806], grad_fn=<AddBackward0>)

In [9]:
fnn = FeatureNN(config=config, name="Feature_NN", input_shape=batch[0].shape, num_units=64, shallow=False)

In [10]:
fnn

FeatureNN(
  (relu): ReLU(inplace=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (_h1): Linear(in_features=64, out_features=64, bias=True)
  (_h2): Linear(in_features=64, out_features=32, bias=True)
  (linear): Linear(in_features=32, out_features=1, bias=True)
  (hidden_layers): Sequential(
    (0): ExU()
    (1): Linear(in_features=64, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=32, bias=True)
  )
)

In [14]:
fnn(batch[0])

tensor([-20471.6133,  -8078.4175, -13975.0645,  ...,  37003.4414,
          7193.8286,   9193.8447], grad_fn=<SqueezeBackward1>)